# Dementia Evaluation

In [6]:
import os, random, time, numpy as np, pandas as pd
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import losses, optimizers, metrics
from tensorflow.keras import Input, Model, layers, callbacks, regularizers, models
from jarvis.train import custom, datasets, params
from jarvis.train.client import Client
from jarvis.utils.general import gpus, overload, tools as jtools

In [7]:
# --- Initialize client
client = Client('/home/mmorelan/proj/dementia/yml/client-3d-96x128.yml', configs={'batch': {'fold': 0}})

In [8]:
# --- Load contrastive model
model = models.load_model("/home/mmorelan/proj/dementia/train/96x128/01/01/model.hdf5", compile=False)

In [9]:
# --- Extract autoencoder model from contrastive model
enc_model = model.layers[4]

In [11]:
# --- Create validation generator
test_train, test_valid = client.create_generators(test=True)

In [12]:
# --- Aggregate all examples
xs = []
ys = []

for x, y in test_valid:
    xs.append(x['dat'])
    ys.append(y['lbl'])

xs = np.concatenate(xs)
ys = np.concatenate(ys)

[ 2021-08-01 13:13:45 ] [====================] 100.000% : Iterating | 000669    

In [13]:
# --- Predict
logits = enc_model.predict(xs)

In [14]:
# --- Calculate metrics
if type(logits) is dict:
    logits = logits['lbl']

# --- Binarize probability predictions using a 0.5 threshold
pred = logits[2]
pred[pred < 0.5] = 0
pred[pred >= 0.5] = 1

df = pd.DataFrame()
df['pred'] = np.squeeze(pred)
df['true'] = np.squeeze(ys)
df['corr'] = df['pred'] == df['true']

print("Accuracy:", df['corr'].mean())

Accuracy: 0.5575485799701047
